In [1]:
import os
import time
import datetime as dt
import glob
from multiprocessing import Pool
import traceback

from tqdm.notebook import tqdm, trange
import numpy as np


In [2]:
lag_dir =r"D:\SIA_Weddell_Sea"
years_range = range(2015,2016)

start_files = []
for year in tqdm(years_range):
    prop_files = sorted(glob.glob(f'{lag_dir}/sic/{year}/sic_????????_????????.npz'))
    if len(prop_files) > 0:
        print(year, prop_files[0])
        start_files.append(prop_files[0])
    else:
        print(year)


  0%|          | 0/1 [00:00<?, ?it/s]

2015 D:\SIA_Weddell_Sea/sic/2015\sic_20150301_20150301.npz


In [5]:
max_prop_duration = 2199 #2199

for start_file in tqdm(start_files):
    date0 = dt.datetime.strptime(os.path.basename(start_file).split('_')[1], '%Y%m%d')
    date1 = dt.datetime.strptime(os.path.basename(start_file).split('_')[2], '%Y%m%d.npz')
    prop_duration = (date1 - date0).days
    if prop_duration >= max_prop_duration:
        continue
    #print('\n\n\n', date0, date1, prop_duration, start_file)
    start_date = date1
    src_npz_file = start_file
    c0 = None
    for i in trange(1, max_prop_duration - prop_duration, 1):
        dst_date = start_date + dt.timedelta(i)
            
        dst_npz_file = f'{lag_dir}/sic/{date0.strftime(f"%Y/sic_%Y%m%d")}_{dst_date.strftime("%Y%m%d")}.npz'
        if os.path.exists(dst_npz_file):
            src_npz_file = dst_npz_file
            c0 = None
            #print(f'{dst_npz_file} exists')
            continue

        mesh_file = f'{lag_dir}/mesh/{dst_date.strftime(f"%Y/mesh_%Y%m%d")}.npz'
        #print(f'Generate {dst_npz_file} with {mesh_file}')
        if not os.path.exists(mesh_file):
            print(f'Error: {mesh_file} does not exist')
            break

        # load mesh
        with np.load(mesh_file) as data:
            src2dst = data['src2dst']
            weights = data['weights']
            ar = data['ar']
        ar[ar == 0] = 0.01

        # load source concentration
        if c0 is None:
            c0 = np.load(src_npz_file)['c']

            # option? 
            #c0_raw = np.load(src_npz_file)['c']    
            #c0 = np.zeros(src2dst[:,1].max()+1, dtype=c0_raw.dtype)
            #c0[:c0_raw.shape[0]] = c0_raw

        # propagate SIC
        c1 = np.zeros(src2dst[:,1].max()+1)
        np.add.at(c1, src2dst[:,1], c0[src2dst[:,0]] * weights)
        c1 /= ar
        c1 = np.clip(c1, 0, 100)

        # cap by observed SIC
        sic_file = dst_date.strftime(f'{lag_dir}/sic/%Y/sic_%Y%m%d.npz')
        if not os.path.exists(sic_file):
            print(f'Error: {sic_file} does not exist')
            break
        c1_obs = np.load(sic_file)['c']
        c1[c1 > c1_obs] = c1_obs[c1 > c1_obs]

        # reuse the propagated fraction
        c0 = c1

        # save the file
        #print(dst_npz_file)
        np.savez(dst_npz_file, c=c1.astype(np.float32))




  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2198 [00:00<?, ?it/s]

Error: D:\SIA_Weddell_Sea/sic/2016/sic_20160101.npz does not exist


In [6]:
sic_min_file = r"D:\SIA_Weddell_Sea\sic\2015\sic_20150301_20150301.npz"
with np.load(sic_min_file) as f:
    myi = dict(**f)
    
mesh_file = r"D:\SIA_Weddell_Sea\mesh\2015\mesh_20150301.npz"
with np.load(mesh_file) as f:
    m = dict(**f)

sic_file = r"D:\SIA_Weddell_Sea\sic\2015\sic_20150301.npz"
with np.load(sic_file) as f:
    sic = dict(**f)

print(myi['c'].shape, m['t'].shape, sic['c'].shape)

(9611,) (9611, 3) (9611,)
